In [4]:
import yfinance as yf
from datetime import datetime, timedelta

def calculate_normalized_volume_change(ticker, target_date):
    try:
        # Ensure target_date is in correct format
        target_date = datetime.strptime(target_date, "%Y-%m-%d").date()
        
        # Fetch data for the last 7 days + the target date
        start_date = target_date - timedelta(days=7)
        end_date = target_date + timedelta(days=1)  # Include the target date
        
        # Fetch historical data
        stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date)
        
        # Ensure data for the target date is available
        if target_date not in stock_data.index.date:
            raise ValueError(f"No data available for {ticker} on {target_date}")
        
        # Extract volume for the target date
        target_date_volume = stock_data.loc[stock_data.index.date == target_date, 'Volume'].iloc[0]
        
        # Calculate weekly average volume (excluding target date)
        weekly_data = stock_data.loc[stock_data.index.date != target_date, 'Volume']
        weekly_avg_volume = weekly_data.mean()
        
        # Calculate Normalized Relative Volume Change
        if weekly_avg_volume > 0:
            normalized_rvc = abs((target_date_volume - weekly_avg_volume) / weekly_avg_volume * 100)
        else:
            normalized_rvc = 0  # Avoid division by zero
        
        return normalized_rvc
    except Exception as e:
        return {"Error": str(e)}

# Example usage
ticker_date_pairs = [
    {"ticker": "AAPL", "date": "2025-01-22"},
    {"ticker": "MSFT", "date": "2025-01-22"},
    {"ticker": "TSLA", "date": "2025-01-22"}
]

# Calculate and output Normalized Volume Change for each
for item in ticker_date_pairs:
    result = calculate_normalized_volume_change(item['ticker'], item['date'])
    if isinstance(result, dict) and "Error" in result:
        print(f"Error for {item['ticker']}: {result['Error']}")
    else:
        print(f"{item['ticker']}: Normalized Volume Change: {result:.2f}")


AAPL: Normalized Volume Change: 7.78
MSFT: Normalized Volume Change: 27.51
TSLA: Normalized Volume Change: 26.56


In [9]:
import yfinance as yf
from datetime import datetime, timedelta

def detect_price_anomalies(ticker, target_date):
    try:
        # Ensure target_date is in correct format
        target_date = datetime.strptime(target_date, "%Y-%m-%d").date()
        
        # Fetch data for the last 7 days + the target date
        start_date = target_date - timedelta(days=7)
        end_date = target_date + timedelta(days=1)  # Include the target date
        stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date)
        
        # Ensure data for the target date is available
        if target_date not in stock_data.index.date:
            raise ValueError(f"No data available for {ticker} on {target_date}")
        
        # Extract data for the target date
        target_day = stock_data.loc[stock_data.index.date == target_date]
        target_open = target_day['Open'].iloc[0]
        target_close = target_day['Close'].iloc[0]
        
        # Calculate percentage price change
        absolute_change = target_close - target_open
        percentage_change = (absolute_change / target_open) * 100
        
        return {
            "Ticker": ticker,
            "Percentage Change (%)": percentage_change
        }
    except Exception as e:
        return {"Ticker": ticker, "Error": str(e)}

# Example usage for multiple stocks
tickers = ["AAPL", "MSFT", "TSLA", "AMZN", "GOOGL"]
target_date = "2025-01-22"

# Analyze all tickers
results = [detect_price_anomalies(ticker, target_date) for ticker in tickers]

# Filter out errors and sort by percentage change
valid_results = [result for result in results if "Error" not in result]
sorted_results = sorted(valid_results, key=lambda x: x["Percentage Change (%)"], reverse=True)

# Display sorted results
print("\nStocks Sorted by Percentage Price Change")
print("===========================================")
for result in sorted_results:
    print(f"Ticker: {result['Ticker']}, Percentage Change: {result['Percentage Change (%)']:.2f}%")
print("===========================================")



Stocks Sorted by Percentage Price Change
Ticker: MSFT, Percentage Change: 1.97%
Ticker: AAPL, Percentage Change: 1.84%
Ticker: AMZN, Percentage Change: 1.29%
Ticker: GOOGL, Percentage Change: -0.35%
Ticker: TSLA, Percentage Change: -0.41%


In [11]:
def calculate_combined_score(ticker, target_date):
    try:
        # Fetch volume anomaly
        volume_result = calculate_normalized_volume_change(ticker, target_date)
        if isinstance(volume_result, dict) and "Error" in volume_result:
            return {"Ticker": ticker, "Error": volume_result["Error"]}
        
        # Fetch price anomaly
        price_result = detect_price_anomalies(ticker, target_date)
        if "Error" in price_result:
            return {"Ticker": ticker, "Error": price_result["Error"]}
        
        # Metrics
        normalized_volume_change = volume_result
        percentage_price_change = price_result["Percentage Change (%)"]
        
        # Weights for volume and price
        w_v = 1  # Weight for volume
        w_p = 2  # Weight for price
        
        # Calculate combined score
        combined_score = (w_v * normalized_volume_change) + (w_p * percentage_price_change)
        
        return {
            "Ticker": ticker,
            "Normalized Volume Change": normalized_volume_change,
            "Percentage Price Change": percentage_price_change,
            "Combined Score": combined_score
        }
    except Exception as e:
        return {"Ticker": ticker, "Error": str(e)}

# Example usage for multiple stocks
tickers = ["AAPL", "MSFT", "TSLA", "AMZN", "GOOGL"]
target_date = "2025-01-22"

# Analyze all tickers
results = [calculate_combined_score(ticker, target_date) for ticker in tickers]

# Filter out errors and sort by combined score
valid_results = [result for result in results if "Error" not in result]
sorted_results = sorted(valid_results, key=lambda x: x["Combined Score"], reverse=True)

# Display sorted results
print("\nStocks Sorted by Combined Score")
print("===========================================")
for result in sorted_results:
    print(f"Ticker: {result['Ticker']}, Combined Score: {result['Combined Score']:.2f}")
    print(f" - Normalized Volume Change: {result['Normalized Volume Change']:.2f}")
    print(f" - Percentage Price Change: {result['Percentage Price Change']:.2f}%")
    print("-------------------------------------------")



Stocks Sorted by Combined Score
Ticker: MSFT, Combined Score: 31.46
 - Normalized Volume Change: 27.51
 - Percentage Price Change: 1.97%
-------------------------------------------
Ticker: TSLA, Combined Score: 25.74
 - Normalized Volume Change: 26.56
 - Percentage Price Change: -0.41%
-------------------------------------------
Ticker: AMZN, Combined Score: 22.40
 - Normalized Volume Change: 19.82
 - Percentage Price Change: 1.29%
-------------------------------------------
Ticker: AAPL, Combined Score: 11.46
 - Normalized Volume Change: 7.78
 - Percentage Price Change: 1.84%
-------------------------------------------
Ticker: GOOGL, Combined Score: 7.02
 - Normalized Volume Change: 7.71
 - Percentage Price Change: -0.35%
-------------------------------------------
